In [118]:
import pandas as pd

import nltk
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer
# nltk.download('stopwords')

import numpy as np
import re

import operator

import pickle

In [94]:
stopwords = set(stopwords.words('english'))

In [95]:
print(stopwords)

{'but', 'doing', 'then', 'because', 'll', 'yourself', "wouldn't", 'does', 'why', 'be', 'up', 'any', "shan't", 'she', 'when', 'o', "haven't", 'what', 'these', 'being', 'shan', 'all', 'ourselves', 'aren', 'he', 'between', 'only', 'it', 'other', 've', 'mightn', 'themselves', 'of', 'have', "didn't", 'for', 'to', 'at', 'each', 'myself', 'were', 'don', 'yours', 'been', "couldn't", 'a', 'shouldn', 'ours', 'those', 'me', "doesn't", 'more', 'himself', 'into', "that'll", "needn't", 'over', 'from', "should've", 'couldn', "weren't", 'some', 'again', 'am', 'in', 'do', 'now', 'here', 'their', 'whom', 'her', 'doesn', 'ma', 'own', 'should', 'them', 'ain', 'we', 'just', 'was', "shouldn't", "hasn't", 'yourselves', 'will', 'this', 'having', 'most', 'nor', 'are', 'not', 'after', "you've", "it's", 'its', 'no', 'off', 'needn', 'wasn', 'hadn', 'before', 'that', "you'll", 'won', 'few', 'with', 'weren', 'my', 'until', 'theirs', 'there', 'wouldn', 'herself', 'y', "mustn't", 'about', 'by', 'such', 'further', 'an

In [96]:
# Einlesen und Rückgabe des Korpus
def readCorpus(reviews):
	content = [', '.join(reviews)]
	# with open(reviews) as f:
	# 	content = f.readlines()
	return content

In [97]:
# Erstellen und Rückgabe eines einfachen Index
# Dokumente sollen tokenisiert und die tokens von nicht Buchstaben gesäubert und in Kleinschreibung umgewandelt werden: z.B. ['abwarten', 'und', 'tee', 'trinken']
def indexCorpus(corpus):
    stemmer = SnowballStemmer('english')
    index = []
    for line in corpus:
        linei = []
        items = line.split(' ')
        for item in items:
            itemt = re.sub(r'\W+', '', item).lower()
            if itemt not in stopwords:
                itemts = stemmer.stem(itemt)
                linei.append(itemts)
        index.append(linei)
    return index

In [32]:
# def indexCorpusWSS(corpus, stopwords):
# 	stemmer = SnowballStemmer('english')
# 	index = []
# 	pos = 0
# 	for c in corpus:
# 		cindex = set()
# 		terms = c.split(' ')
# 		for t in terms:
# 			tt = re.sub(r'\W+', '', t).lower()
# 			if tt not in stopwords:
# 				tts = stemmer.stem(tt)
# 				if tts not in cindex:
# 					cindex.add(tts)
# 		index[pos] = cindex
# 		pos += 1
# 	return index

In [98]:
data = pd.read_csv('./data/steam_game_reviews.csv')

In [99]:
data.head()

,Unnamed: 0,title,review_count,recommendation_count,recommendation_ratio,avg_playtime,reviews_combined
0,0,ACE COMBAT™ 7: SKIES UNKNOWN,8,8.0,1.000000,19.750000,Oh man it's been at least a decade since Ace C...
1,1,ARK: Survival Evolved,130,25.0,0.192308,1007.007692,Before i start the review THE GAMEPLAY OF THIS...
2,2,ASTRONEER,1436,1368.0,0.952646,54.400418,i have not played much of this game and i dont...
3,3,Battlefleet Gothic: Armada 2,8,6.0,0.750000,62.750000,Dear Devs thanks for taking your time and maki...
4,4,Beat Saber,6,6.0,1.000000,80.500000,When I pull off a really difficult section I g...


In [100]:
corpus = data.loc[:,['title','reviews_combined']]

In [101]:
print(corpus)

                                            title  \
0                    ACE COMBAT™ 7: SKIES UNKNOWN   
1                           ARK: Survival Evolved   
2                                       ASTRONEER   
3                    Battlefleet Gothic: Armada 2   
4                                      Beat Saber   
5                                     Cold Waters   
6                                Dead by Daylight   
7   Divinity: Original Sin 2 - Definitive Edition   
8                           Don't Starve Together   
9                          Euro Truck Simulator 2   
10    Expansion - Hearts of Iron IV: Man the Guns   
11                                       Factorio   
12                           Farming Simulator 19   
13                          Football Manager 2019   
14                                     Foundation   
15                                    GOD EATER 3   
16                                    Garry's Mod   
17                             Grand Theft Aut

In [102]:
# index = indexCorpus(corpus['reviews_combined'])

In [120]:
# with open('./data/index.pickle', 'wb') as handle:
#     pickle.dump(index, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [121]:
with open('./data/index.pickle', 'rb') as handle:
    index = pickle.load(handle)

In [82]:
# Berechnung und Rückgabe des Rangierungsscore Nr 1
# Je mehr Suchbegriffe in einem Dokument vorkommen, desto relevanter ist das Dokument.
def calculateScore1(index, query):
    scores = []
    qterms = query.split(' ')
    for ind in index:
        score = 0
        for t in qterms:
            for dt in ind:
                if t == dt:
                    score = score + 1
                    break
        scores.append(score)
    return scores

# Berechnung und Rückgabe des Rangierungsscore Nr 2
# Je haeufiger die Suchbegriffe in einem Dokument vorkommen, desto relevanter ist das Dokument.
def calculateScore2(index, query):
    scores = []
    qterms = query.split(' ')
    for ind in index:
        score = 0
        for t in qterms:
            for dt in ind:
                if t == dt:
                    score = score + 1
        scores.append(score)
    return scores

# Berechnung und Rückgabe des Rangierungsscore Nr 3
# Je frueher ein Suchbegriff in einem Dokument vorkommt, desto relevanter ist das Dokument. Maximal Score ist 10.0.
def calculateScore3(index, query):
    scores = []
    qterms = query.split(' ')
    for ind in index:
        score = 10.0
        for t in qterms:
            for pos, dt in enumerate(ind):
                if t == dt:
                    if pos < score:
                        score = pos + 1
                    break
        fsc = 10.0 / (score)
        scores.append(max(1.0, fsc))
    return scores

# Berechnen aller 3 Rangierungsscores zwischen der Query und den Dokumenten. Der finale Score wird durch die Multiplikation aller 3 Scores berechnet.
def calculateScores(index, query):
    tmp = [calculateScore1(index, query), calculateScore2(index, query), calculateScore3(index, query)]
    final_scores = {}
    for p, ind in enumerate(index):
        final_scores[p] = tmp[0][p] * tmp[1][p] * tmp[2][p]
    return final_scores

In [103]:
# calculateScores(index, "product")

In [85]:
# Erstellen des Rankings und Ausgabe der topn Sprichwoerter mit dem jeweiligen Score.
def evaluateScores(scores, corpus, topn):
	sorted_x = sorted(scores.items(), key=operator.itemgetter(1), reverse=True)
	topnitems = sorted_x[0:topn]
	result = ''
	for item in topnitems:
		if item[1] > 0.0:
			result = result + '\n' + corpus['title'][item[0]].strip() + ' --> Score: ' + str(item[1])
	return result

In [122]:
print('Antwort 1: Die 5 am höchsten bewerteten Games für den Suchterm sind:\n {}'.format(evaluateScores(calculateScores(index, "best game ever"), corpus, 5)))

Antwort 1: Die 5 am höchsten bewerteten Games für den Suchterm sind:
 
Grand Theft Auto V --> Score: 2407560.0
PLAYERUNKNOWN'S BATTLEGROUNDS --> Score: 456375.0
Rocket League® --> Score: 341242.5
Rust --> Score: 210765.0
Dead by Daylight --> Score: 92133.0
